In [1]:
from packages.utils import fetch_crude_oil, fetch_crude_oil_brent, fetch_daily_stock, fetch_daily_fx, fetch_daily_stock, get_row_by_date, load_to_s3
import requests
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import s3fs
from datetime import datetime

In [2]:
api_key = os.getenv('AV_API_KEY')
date_format = "%Y-%m-%d"

In [3]:
url = "https://www.alphavantage.co/query"
params = {
"function": "TIME_SERIES_DAILY",
"symbol": "AAPL",
"datatype":"json",
"outputsize":"compact",#compact/full
"apikey": api_key
}

In [4]:
response = requests.get(url, params=params, verify=False)
data = response.json()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.alphavantage.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
df = fetch_daily_stock('AAPL')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.alphavantage.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [6]:
df

,open,high,low,close,volume
2024-04-09,168.7000,170.08,168.3500,169.67,42231444
2024-04-10,168.8000,169.09,167.1100,167.78,49709336
2024-04-11,168.3400,175.46,168.1600,175.04,91070275
2024-04-12,174.2600,178.36,174.2100,176.55,101670886
2024-04-15,175.3600,176.63,172.5000,172.69,73531773
...,...,...,...,...,...
2024-08-23,225.6589,228.22,224.3300,226.84,38677250
2024-08-26,226.7600,227.28,223.8905,227.18,30602208
2024-08-27,225.9950,228.85,224.8900,228.03,35934559
2024-08-28,227.9200,229.86,225.6800,226.49,38052167


In [7]:
specificSeries = get_row_by_date(df, '2024-08-29')
print(specificSeries)

open           230.10
high           232.92
low            228.88
close          229.79
volume    51906297.00
Name: 2024-08-29 00:00:00, dtype: float64


In [8]:
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

In [9]:
load_to_s3(df,'myprojects-ak/dataEngineeringPOC','testdf', aws_access_key_id, aws_secret_access_key)

Data successfully written to s3://myprojects-ak/dataEngineeringPOC/2024-08-30/testdf.parquet


In [10]:

current_date = datetime.now().strftime('%Y-%m-%d')
s3_file_path = f"s3://myprojects-ak/dataEngineeringPOC/{current_date}/testdf.parquet"

df_read = pd.read_parquet(s3_file_path, storage_options={
    "key": aws_access_key_id,
    "secret": aws_secret_access_key,
})

print(df_read.head())

     open    high     low   close     volume
0  168.70  170.08  168.35  169.67   42231444
1  168.80  169.09  167.11  167.78   49709336
2  168.34  175.46  168.16  175.04   91070275
3  174.26  178.36  174.21  176.55  101670886
4  175.36  176.63  172.50  172.69   73531773
